In [ ]:
# Import các thư viện Python cần thiết
import folium
import pandas as pd

# Import các plugin của Folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

# Tải bộ dữ liệu
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(URL)

# Chọn các cột cần thiết và loại bỏ các hàng trùng lặp để lấy danh sách các bãi phóng
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
print("Thông tin các bãi phóng:")
print(launch_sites_df)
print("\n" + "="*50 + "\n")


Thông tin các bãi phóng:
    Launch Site        Lat        Long
0   CCAFS LC-40  28.562302  -80.577356
1  CCAFS SLC-40  28.563197  -80.576820
2    KSC LC-39A  28.573255  -80.646895
3   VAFB SLC-4E  34.632834 -120.610745




In [2]:
# TASK 1: ĐÁNH DẤU TẤT CẢ CÁC BÃI PHÓNG TRÊN BẢN ĐỒ
#------------------------------------------------------------------
print("Đang thực hiện TASK 1: Đánh dấu các bãi phóng...")

# Tạo bản đồ ban đầu, lấy vị trí trung tâm là một trong các bãi phóng
# Ví dụ: Mũi Canaveral, Florida
nasa_coordinate = [28.562302, -80.577356]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Tạo một nhóm đối tượng (FeatureGroup) cho các bãi phóng để dễ quản lý
launch_sites_group = folium.FeatureGroup(name="Launch Sites").add_to(site_map)

# Lặp qua từng bãi phóng trong dataframe và thêm marker vào bản đồ
for index, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    coordinate = [row['Lat'], row['Long']]
    
    # Thêm một vòng tròn xung quanh bãi phóng
    folium.Circle(
        location=coordinate,
        radius=1000,  # bán kính 1000 mét
        color='#0078ff',
        fill=True,
        fill_color='#0078ff'
    ).add_to(launch_sites_group)
    
    # Thêm một marker với tên của bãi phóng
    folium.Marker(
        location=coordinate,
        icon=folium.Icon(color='white', icon_color='black', icon='rocket', prefix='fa'),
        popup=site_name
    ).add_to(launch_sites_group)

print("TASK 1 Hoàn thành. Bản đồ 'site_map' đã được cập nhật.")
print("\n" + "="*50 + "\n")

Đang thực hiện TASK 1: Đánh dấu các bãi phóng...
TASK 1 Hoàn thành. Bản đồ 'site_map' đã được cập nhật.




In [3]:
# TASK 2: ĐÁNH DẤU KẾT QUẢ PHÓNG (THÀNH CÔNG/THẤT BẠI) CHO MỖI BÃI
#------------------------------------------------------------------
print("Đang thực hiện TASK 2: Đánh dấu kết quả phóng...")

# Tạo một đối tượng MarkerCluster để nhóm các marker lại khi zoom xa
marker_cluster = MarkerCluster().add_to(site_map)

# Thêm cột 'marker_color' vào dataframe để xác định màu sắc
# 'green' cho thành công (Class=1), 'red' cho thất bại (Class=0)
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'  # Thành công
    else:
        return 'red'    # Thất bại

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Lặp qua từng vụ phóng và thêm marker vào MarkerCluster
for index, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Site: {record['Launch Site']}\nOutcome: {'Success' if record['class']==1 else 'Failure'}"
    ).add_to(marker_cluster)
    
print("TASK 2 Hoàn thành. Bản đồ 'site_map' đã được cập nhật với các cụm marker.")
print("\n" + "="*50 + "\n")

Đang thực hiện TASK 2: Đánh dấu kết quả phóng...
TASK 2 Hoàn thành. Bản đồ 'site_map' đã được cập nhật với các cụm marker.




In [4]:
# TASK 3: TÍNH TOÁN VÀ VẼ KHOẢNG CÁCH
#------------------------------------------------------------------
print("Đang thực hiện TASK 3: Thêm tính năng tính khoảng cách...")

# Thêm plugin hiển thị tọa độ khi di chuyển chuột trên bản đồ
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Tọa độ:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(site_map)


# Hàm tính khoảng cách Haversine giữa 2 điểm tọa độ
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # bán kính Trái Đất theo km

    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Ví dụ: Tính khoảng cách từ bãi phóng KSC LC-39A đến đường bờ biển gần nhất
# Tọa độ này được tìm thấy bằng cách dùng plugin MousePosition trên bản đồ
launch_site_lat = 28.573255
launch_site_lon = -80.646895
coastline_lat = 28.57325
coastline_lon = -80.6065

# Tính khoảng cách
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Tạo marker cho điểm bờ biển
distance_marker = folium.Marker(
   [coastline_lat, coastline_lon],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>',
       )
   )

# Vẽ đường nối giữa bãi phóng và bờ biển
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)

# Thêm vào bản đồ
site_map.add_child(distance_marker)
site_map.add_child(lines)

# Thêm LayerControl để có thể bật/tắt các lớp
folium.LayerControl().add_to(site_map)

print("TASK 3 Hoàn thành. Bản đồ đã có đường kẻ và khoảng cách mẫu.")
print("\n" + "="*50 + "\n")

Đang thực hiện TASK 3: Thêm tính năng tính khoảng cách...
TASK 3 Hoàn thành. Bản đồ đã có đường kẻ và khoảng cách mẫu.




In [ ]:
# HIỂN THỊ BẢN ĐỒ

site_map.save("spacex_launch_sites_map.html")
print("Bản đồ đã được lưu vào file spacex_launch_sites_map.html")

# Hiển thị bản đồ (chạy trong Jupyter)
site_map

Bản đồ đã được lưu vào file spacex_launch_sites_map.html
